<a href="https://colab.research.google.com/github/ced-sys/.py/blob/main/Quantum_BERT_LLm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate datasets trl peft bitsandbytes
!pip install wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import(
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    pipeline
)

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
import json
import random
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import re

In [ ]:
import bitsandbytes
print(bitsandbytes.__version__)

0.47.0


In [ ]:
class QuantumPhysicsLLM:
  """A Generative LLM specifically designed to teach quantum physics"""
  def __init__(self, model_name="microsoft/DialoGPT-small", use_lora=True):
    """
    Initialize the quantum physics teaching LLM

    Args:
    Model_name: Base model to use (small for colab)
    use_lora: Whether to us LoRA for efficient fine-tuning"""

    self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f' Using device: {self.device}')

    #Load tokenizer
    self.tokenizer=AutoTokenizer.from_pretrained(model_name)
    if self.tokenizer.pad_token is None:
      self.tokenizer.pad_token=self.tokenizer.eos_token

      #Configure for memory efficient
      if torch.cuda.is_available():
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )
      else:
        quantization_config=None


      #Load Base model
      self.model=AutoModelForCasualLM.from_pretrained(
          model_name,
          quantization_config=quantization_config,
          device_map='auto' if torch.cuda.is_available() else None,
          torch_dtype=torch.float16 if torch.cuda.is_availabe() else torch.float32
      )

      #Apply LoRA for efficient fine-tuning
      if use_lora:
        lora_config=LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,

        )